In [2]:
import pandas as pd
import os
import numpy as np

# Drug Screen

In [3]:
drug = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results/drug_list.csv')
drug['cmap_drug'] = [str(i)+'_'+j for i,j in zip (drug['instance_id'].tolist(), drug['cmap_name'].tolist())]

In [31]:
drug.head()

,instance_id,batch_id,cmap_name,INN1,concentration (M),duration (h),cell2,array3,perturbation_scan_id,vehicle_scan_id4,scanner,vehicle,vendor,catalog_number,catalog_name,cmap_drug
0,1,1,metformin,INN,1.000000e-05,6,MCF7,HG-U133A,EC2003090503AA,EC2003090502AA,HP GeneArray Scanner,medium,Sigma-Aldrich,D5035,"1,1-dimethylbiguanide hydrochloride",1_metformin
1,2,1,metformin,INN,1.000000e-05,6,MCF7,HG-U133A,EC2003090504AA,EC2003090502AA,HP GeneArray Scanner,medium,Sigma-Aldrich,D5035,"1,1-dimethylbiguanide hydrochloride",2_metformin
2,3,1,metformin,INN,1.000000e-07,6,MCF7,HG-U133A,EC2003090505AA,EC2003090502AA,HP GeneArray Scanner,medium,Sigma-Aldrich,D5035,"1,1-dimethylbiguanide hydrochloride",3_metformin
3,4,1,metformin,INN,1.000000e-03,6,MCF7,HG-U133A,EC2003090506AA,EC2003090502AA,HP GeneArray Scanner,medium,Sigma-Aldrich,D5035,"1,1-dimethylbiguanide hydrochloride",4_metformin
4,21,2,phenformin,INN,1.000000e-05,6,MCF7,HG-U133A,EC2003091104AA,EC2003091102AA,HP GeneArray Scanner,medium,Sigma-Aldrich,P7045,phenformin hydrochloride,21_phenformin


In [32]:
ds = 'DS4A'
sample = 'DS4A.1'
# full list of cell_type depend on txt file names from drug_screen_parent_dir
cell_type = 'Malignant'

In [33]:
drug_perturb_parent_dir = '/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Output_OVR_PerturbationNetwork/'+ds+'/'+sample+'/DGE_dec'
perturb_cmap = dict()
for i in os.listdir(drug_perturb_parent_dir):
    if cell_type in i:
        new_dir = drug_perturb_parent_dir + '/' + i
        perturb_cmap[i] = []
        os.chdir(new_dir)
        for j in os.listdir():
            cmap = int(j.split('.')[0])
            perturb_cmap[i].append(cmap)

In [34]:
perturb_cmap

{'Malignant_#_INV': [5360,
  5339,
  993,
  6944,
  5312,
  1607,
  992,
  6216,
  5306,
  1080,
  1103,
  7176,
  621,
  624,
  2807,
  6205,
  1555,
  5284,
  5355,
  952,
  1044,
  1077,
  5322,
  6963,
  6969,
  1651,
  6234,
  1598,
  1075,
  6965,
  2338,
  836,
  1004,
  5298,
  1049,
  1043,
  5294,
  5315,
  986,
  961,
  5336,
  5350,
  5342,
  958,
  1051,
  1053,
  1001,
  7166,
  1056,
  5344,
  1647,
  6985,
  5358,
  882,
  5331,
  987,
  1627,
  161,
  5349,
  1064,
  5326,
  6967,
  1626,
  5283,
  5333,
  6925,
  5359,
  6966,
  7407,
  1042,
  164,
  1112,
  832,
  6975,
  1050,
  1662,
  6973,
  1613,
  6978,
  7417,
  6249,
  944,
  6946,
  7405,
  5357,
  1059,
  1045,
  1041,
  5337,
  6972,
  6926,
  7406,
  999,
  5340,
  1653,
  5345,
  6976,
  5317,
  622,
  1055],
 'Malignant_#_POS': [2967,
  1057,
  2792,
  6765,
  7182,
  704,
  7094,
  7349,
  7036,
  1502,
  6261,
  7419,
  2755,
  4794,
  1501,
  7438,
  2079,
  7037,
  7186,
  7100,
  1708,
  1115,
  1

In [35]:
drug_screen_parent_dir = '/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Output_OVR_DrugScreen/'+ds+'/'+ sample+'/DGE_dec'
ct_drug_screen = pd.read_csv(drug_screen_parent_dir+'/'+cell_type+'.txt', sep = '\t', header=None)
ct_drug_screen.columns = ['Enrichment_score','P-value','Adj-p']
ct_drug_screen['Drug'] = drug['cmap_name']
ct_drug_screen['CMAP_instance'] = drug['instance_id']
ct_drug_screen = ct_drug_screen[['CMAP_instance','Drug','Enrichment_score','P-value','Adj-p']]

In [36]:
ct_drug_screen.head()

,CMAP_instance,Drug,Enrichment_score,P-value,Adj-p
0,1,metformin,0.293,0,0
1,2,metformin,0.000,1,1
2,3,metformin,0.273,0,0
3,4,metformin,0.000,1,1
4,21,phenformin,0.000,1,1


In [37]:
# note, maybe not all cell types of all samples have INV and POS drugs?
ct_INV_es = ct_drug_screen[ct_drug_screen['CMAP_instance'].isin(perturb_cmap[cell_type+'_#_INV'])]
ct_INV_es = ct_INV_es.sort_values('Enrichment_score', ascending=False).reset_index(drop=True)
ct_POS_es = ct_drug_screen[ct_drug_screen['CMAP_instance'].isin(perturb_cmap[cell_type+'_#_POS'])]
ct_POS_es = ct_POS_es.sort_values('Enrichment_score').reset_index(drop=True)

In [38]:
ct_POS_es.head()

,CMAP_instance,Drug,Enrichment_score,P-value,Adj-p
0,614,monastrol,-0.638,0,0
1,1135,minocycline,-0.541,0,0
2,5297,tacrine,-0.537,0,0
3,1108,"12,13-EODE",-0.521,0,0
4,1073,genistein,-0.518,0,0


In [39]:
enrich_sample_dir = '/share/fsmresfiles/SpatialT/website_drug_target/Enrichment/'+ds+'/'+sample+'/'
if not os.path.exists(enrich_sample_dir):
    os.makedirs(enrich_sample_dir)

In [40]:
ct_POS_es.to_csv(enrich_sample_dir+cell_type+'_POS_es.csv', index=False)
ct_INV_es.to_csv(enrich_sample_dir+cell_type+'_INV_es.csv', index=False)

# PPI

In [41]:
gene = pd.read_table('/share/fsmresfiles/SpatialT/drug-target-results/HGNC.tsv')
gene = gene[['NCBI Gene ID(supplied by NCBI)','Approved symbol']]
gene.columns = ['num','name']
lst = []
for i in gene['num'].tolist():
    if pd.isna(i) != True:
        lst.append(int(i))
    else:
        lst.append(i)
gene['num'] = lst

In [42]:
gene.head()

,num,name
0,100861506.0,LINC00903
1,2457.0,FRA14B
2,111082990.0,LINC02157
3,2467.0,FRA18A
4,104355133.0,ABHD15-AS1


In [43]:
gene_dict = gene.set_index('num').to_dict()['name']

In [44]:
ppi = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Output_OVR_PPI/'+ds+'/'+sample+'/DGE_dec/'+cell_type+ '.txt', sep='\t', header = None)
# not alll files are transferrered over from b1131
log2fc = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-b1131/dec/'+ds+'/'+sample+'/DGE_dec/'+cell_type+'.txt', sep='\t')

In [45]:
def node_edge_dfs(ppi, fc):
    ppi_df = pd.DataFrame()
    ppi_df['Source'] = [gene_dict[i] for i in ppi[0].tolist()]
    ppi_df['Target'] = [gene_dict[i] for i in ppi[1].tolist()]
    ppi_genes = set(ppi_df['Source'].tolist() + ppi_df['Target'].tolist())
    ppi_nodes = fc[fc['gene'].isin(ppi_genes)]
    for i in ppi_genes: 
        if i not in fc['gene'].tolist():
            missing = pd.DataFrame({
                "gene": [i],
                "stat": [0],
                "pval": [1],
                "qval": [1],
            }, index=[0])
            ppi_nodes = ppi_nodes.append(missing)
    ppi_nodes['ID'] = ppi_nodes['gene']
    ppi_nodes['Label'] = ppi_nodes['ID']
    ppi_nodes['abs_stat'] = [abs(i) for i in ppi_nodes['stat'].tolist()]
    ppi_nodes['sign'] = [np.sign(i) for i in ppi_nodes['stat'].tolist()]
    ppi_nodes = ppi_nodes[['ID','Label','stat','abs_stat','sign']]
    return ppi_df, ppi_nodes

In [46]:
ppi_df,ppi_nodes = node_edge_dfs(ppi, log2fc)

/tmp/ipykernel_1791/2992167295.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppi_nodes['ID'] = ppi_nodes['gene']
/tmp/ipykernel_1791/2992167295.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppi_nodes['Label'] = ppi_nodes['ID']
/tmp/ipykernel_1791/2992167295.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [47]:
ppi_df.head()

,Source,Target
0,COL1A2,DDR1
1,COL1A2,COL1A1
2,COL1A2,DCN
3,COL1A2,LUM
4,COL1A2,MMP9


In [48]:
ppi_nodes.head()

,ID,Label,stat,abs_stat,sign
0,KRT19,KRT19,3.994943,3.994943,1.0
2,CALML5,CALML5,3.761550,3.761550,1.0
4,KRT8,KRT8,3.452473,3.452473,1.0
7,KRT18,KRT18,3.295043,3.295043,1.0
8,ELF3,ELF3,3.289746,3.289746,1.0


In [49]:
ppi_sample_dir = '/share/fsmresfiles/SpatialT/website_drug_target/PPI/'+ds+'/'+sample+'/'
if not os.path.exists(ppi_sample_dir):
    os.makedirs(ppi_sample_dir)

In [50]:
ppi_nodes.to_csv(ppi_sample_dir+cell_type+'_ppi_nodes.csv', index=False)
ppi_df.to_csv(ppi_sample_dir+cell_type+'_ppi.csv', index=False)

# Perturbation

In [51]:
# full cmap_isntance list = all instances from perturb_cmap
cmap_instance = 1045
direction = 'INV'
cmap_drug = drug[drug['instance_id'] == cmap_instance]['cmap_drug'].tolist()[0]

In [52]:
perturb = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Output_OVR_PerturbationNetwork/'+ds+'/'+sample+'/DGE_dec/'+cell_type+'_#_'+direction+'/'+str(cmap_instance)+'.txt', sep = '\t', header=None)

In [53]:
perturb.head()

,0,1,2,3
0,51078,0.527797,2.0,-1.02
1,2969,0.941104,22.5,-0.81
2,51042,0.811000,35.0,-0.77
3,79035,0.535318,38.5,-0.75
4,29078,0.596571,45.5,-0.73


In [192]:
perturb.columns = ['gene_num','log2fc','rank','exp_z']
perturb['Target'] = [gene_dict[i] for i in perturb['gene_num'].tolist()]
perturb['Source'] = cmap_drug
perturb = perturb[['Source','Target','log2fc','exp_z']]
perturb['abs_log2fc'] = [abs(i) for i in perturb['log2fc'].tolist()]
perturb['sign_log2fc'] = [np.sign(i) for i in perturb['log2fc'].tolist()]
perturb = perturb.sort_values('exp_z')
perturb = pd.concat([perturb[:30],perturb.tail(30)])

In [193]:
perturb.head()

,Source,Target,log2fc,exp_z,abs_log2fc,sign_log2fc
0,1045_sirolimus,THAP4,0.527797,-1.02,0.527797,1.0
1,1045_sirolimus,GTF2I,0.941104,-0.81,0.941104,1.0
2,1045_sirolimus,ZNF593,0.811000,-0.77,0.811000,1.0
3,1045_sirolimus,NABP2,0.535318,-0.75,0.535318,1.0
4,1045_sirolimus,NDUFAF4,0.596571,-0.73,0.596571,1.0


In [194]:
perturb_nodes = perturb[['Target', 'log2fc','sign_log2fc','abs_log2fc']]
perturb_nodes['Id'] = perturb_nodes['Target']
perturb_nodes['Label'] = perturb_nodes['Target']
perturb_nodes = perturb_nodes[['Id','Label','log2fc','sign_log2fc','abs_log2fc']]
drug_row = pd.DataFrame({'Id':[perturb['Source'].unique().tolist()[0]],
                         'Label':[perturb['Source'].unique().tolist()[0]],
                         'log2fc':[2],
                         'sign_log2fc':[0],
                         'abs_log2fc':[2]})
perturb_nodes = pd.concat([drug_row, perturb_nodes])

/tmp/ipykernel_31749/1568994689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perturb_nodes['Id'] = perturb_nodes['Target']


In [195]:
perturb_nodes.head()

,Id,Label,log2fc,sign_log2fc,abs_log2fc
0,1045_sirolimus,1045_sirolimus,2.000000,0.0,2.000000
0,THAP4,THAP4,0.527797,1.0,0.527797
1,GTF2I,GTF2I,0.941104,1.0,0.941104
2,ZNF593,ZNF593,0.811000,1.0,0.811000
3,NABP2,NABP2,0.535318,1.0,0.535318


In [196]:
perturb_sample_dir = '/share/fsmresfiles/SpatialT/website_drug_target/Perturbation/'+ds+'/'+sample+'/'+direction+'/'
if not os.path.exists(perturb_sample_dir):
    os.makedirs(perturb_sample_dir)

In [197]:
perturb.to_csv(perturb_sample_dir+cell_type+'_'+str(cmap_instance)+'_perturb.csv',index=False)
perturb_nodes.to_csv(perturb_sample_dir+cell_type+'_'+str(cmap_instance)+'_perturb_nodes.csv',index=False)

# Gernerating PPI from interactome - exploratory

In [101]:
# clean hgnc genes without gene number
from math import isnan
clean_hgnc_dict = filter(lambda k: not isnan(k), gene_dict)
clean_hgnc_dict = {k: gene_dict[k] for k in gene_dict if not isnan(k)}
clean_hgnc_dict = {int(k):v for k,v in clean_hgnc_dict.items()}

In [102]:
clean_hgnc_dict

{100861506: 'LINC00903',
 2457: 'FRA14B',
 111082990: 'LINC02157',
 2467: 'FRA18A',
 104355133: 'ABHD15-AS1',
 109729137: 'MEF2C-AS2',
 56115: 'PCDHG@',
 260330: 'ALPPP',
 161003: 'STOML3',
 3369: 'HVBS7',
 1055: 'CECR',
 1714: 'DGCR',
 100873962: 'LINC00278',
 111082988: 'LINC02155',
 6693: 'SPN',
 3230: 'HOXD@',
 2437: 'FRA10G',
 100873937: 'GRM7-AS1',
 3655: 'ITGA6',
 2903: 'GRIN2A',
 109729136: 'LILRB1-AS1',
 5782: 'PTPN12',
 116435295: 'CALHM6-AS1',
 111064650: 'C19orf85',
 392392: 'OR1K1',
 26531: 'OR11A1',
 29993: 'PACSIN1',
 56159: 'TEX11',
 51222: 'ZNF219',
 64806: 'IL25',
 11027: 'LILRA2',
 55342: 'STRBP',
 7748: 'ZNF195',
 83999: 'KREMEN1',
 140691: 'TRIM69',
 167465: 'ZNF366',
 57541: 'ZNF398',
 267002: 'PGBD2',
 142679: 'DUSP19',
 29091: 'STXBP6',
 11274: 'USP18',
 4221: 'MEN1',
 4306: 'NR3C2',
 5669: 'PSG1',
 7015: 'TERT',
 7164: 'TPD52L1',
 7248: 'TSC1',
 5136: 'PDE1A',
 2566: 'GABRG2',
 5786: 'PTPRA',
 111064648: 'SMIM36',
 1816: 'DRD5',
 6677: 'SPAM1',
 440533: 'PSG8',

In [103]:
interactome = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Analysis_PPI/interactome.tsv', sep='\t')

In [104]:
# map interactome genes to gene name and remove those that can't be mapped to hgnc gene names
unmappable_pro = []
Protein_A_Name = []
Protein_B_Name = []

for i in interactome['#Protein A'].tolist():
    if i in clean_hgnc_dict.keys(): 
        Protein_A_Name.append(clean_hgnc_dict[i] )
    else:
        if i not in unmappable_pro: 
            unmappable_pro.append(i)
            print(i)
        Protein_A_Name.append('drop')

for i in interactome['Protein B'].tolist():
    if i in clean_hgnc_dict.keys(): 
        Protein_B_Name.append(clean_hgnc_dict[i] )
    else:
        if i not in unmappable_pro: 
            unmappable_pro.append(i)
            print(i)
        Protein_B_Name.append('drop')

244
4029
5171
7795
9503
10638
11217
23285
26148
55747
79686
80761
114299
142937
153684
155060
285464
389834
400590
554045
643752
283970
728932
338809
445347
554223
729603
727738
100294715
441520
1283
100287428
653219
221016
348738
101243544
100507714
85371
440157
642947
339457
164022
653252
100129534
728392
252842
6025
145757
283011
497048
100127889
100652775
728461
653973
100129885
414927
730441
117153
728071
374467
388289
389523
641590
654348
728936
729368
646938
100862685
284083
445341
653602
100526832
653129
284912
728013
162137
643832
442028
497049
727751
392490
554206
652276


In [105]:
len(unmappable_pro)

83

In [106]:
interactome['Protein_A_Name'] = Protein_A_Name
interactome['Protein_B_Name'] = Protein_B_Name

In [107]:
dropa_index = interactome[interactome['Protein_A_Name'] == 'drop'].index.tolist()
dropb_index = interactome[interactome['Protein_B_Name'] == 'drop'].index.tolist()
print(len(dropa_index), len(dropb_index))

101 441


In [108]:
# 538 interactions removed due to unmappable gene number 
for i in dropa_index:
    if i not in dropb_index:
        dropb_index.append(i)
len(dropb_index)

538

In [109]:
clean_interactome = interactome.drop(dropb_index)

In [110]:
# expand ppi results to top 300 for sample ds4a.1 malignant cells
log2fc['abs_stat'] = [abs(i) for i in log2fc['stat'].tolist()]
log2fc_top300 = log2fc.sort_values('abs_stat', ascending=False)[:300]
log2fc_top300_sig = log2fc_top300[log2fc_top300['qval']<0.05]

In [111]:
# find interactome ppis where both proteins are deg proteins
ppi_index_interactome_top300 = []
for i in clean_interactome.iterrows():
    row = i[1]
    if row['Protein_A_Name'] in log2fc_top300_sig['gene'].tolist() and row['Protein_B_Name'] in log2fc_top300_sig['gene'].tolist():
        ppi_index_interactome_top300.append(i[0])
        

In [112]:
ppi_top300 = clean_interactome[clean_interactome.index.isin(ppi_index_interactome_top300)][['#Protein A','Protein B', 'Protein_A_Name','Protein_B_Name']]

In [113]:
# drop ppis where both proteins are the same
same_pro_index_top300 = []
for i in ppi_top300.iterrows():
    row = i[1]
    if row['#Protein A'] == row['Protein B']:
        same_pro_index_top300.append(i[0])

In [114]:
ppi_top300_nored = ppi_top300.drop(same_pro_index_top300)

In [115]:
print(len(ppi_top300_nored),len(ppi_top300))

280 409


In [116]:
# format edges list for output
ppi_top300_nored = ppi_top300_nored[['Protein_A_Name','Protein_B_Name']]
ppi_top300_nored.columns = ['Source','Target']

In [117]:
ppi_top300_nored.head()

,Source,Target
33,A2M,APOD
45,A2M,B2M
50,A2M,CDKN1A
54,A2M,CTSB
67,A2M,GAPDH


In [118]:
# format nodes list for output
ppi_uniq_pro = list(set(ppi_top300_nored['Source'].tolist()) )
for i in ppi_top300_nored['Target'].tolist():
    if i not in ppi_top300_nored['Source'].tolist() and i not in ppi_uniq_pro :
        ppi_uniq_pro.append(i)

In [119]:
ppi_nodes_top300 = pd.DataFrame()
ppi_nodes_top300['ID'] = ppi_uniq_pro
ppi_nodes_top300['Label'] = ppi_uniq_pro

ppi_nodes_top300 = pd.merge(ppi_nodes_top300, log2fc[['gene','stat']], how = 'left', right_on = 'gene', left_on = 'Label')
ppi_nodes_top300 = ppi_nodes_top300.drop('gene',axis = 1)

ppi_nodes_top300['sign'] = [np.sign(i) for i in ppi_nodes_top300['stat'].tolist()]
ppi_nodes_top300['abs_stat'] = [abs(i) for i in ppi_nodes_top300['stat'].tolist()]

In [120]:
ppi_nodes_top300.head()

,ID,Label,stat,sign,abs_stat
0,PTPRF,PTPRF,2.169570,1.0,2.169570
1,HLA-B,HLA-B,-2.651661,-1.0,2.651661
2,COL6A2,COL6A2,-2.711020,-1.0,2.711020
3,CCND1,CCND1,2.643617,1.0,2.643617
4,DSG2,DSG2,1.584564,1.0,1.584564


In [121]:
ppi_top300_nored.head()

,Source,Target
33,A2M,APOD
45,A2M,B2M
50,A2M,CDKN1A
54,A2M,CTSB
67,A2M,GAPDH


In [122]:
ppi_nodes_top300.to_csv(ppi_sample_dir+cell_type+'_ppi_nodes_top300.csv', index=False)
ppi_top300_nored.to_csv(ppi_sample_dir+cell_type+'_ppi_top300.csv', index=False)

In [123]:
uniq_nodes = ppi_top300_nored['Source'].unique().tolist()
append_nodes = [i for i in ppi_top300_nored['Target'].unique().tolist() if i not in ppi_top300_nored['Source'].unique().tolist()]
uniq_nodes.append(append_nodes)

In [124]:
len(uniq_nodes)

105

In [125]:
[ i for i in ppi_nodes_top300['ID'].tolist() if i not in uniq_nodes ]

['APOD',
 'LYZ',
 'TGFBI',
 'AP1M2',
 'EGFL7',
 'SPP1',
 'TNFSF10',
 'HLA-C',
 'NUSAP1',
 'PHB2',
 'FKBP4',
 'GRB7',
 'ITPR1',
 'CTSL',
 'HLA-DQA1',
 'MIF',
 'LST1',
 'IFITM1',
 'PABPC1',
 'CALML5',
 'RRM2',
 'CCL5',
 'SFRP2',
 'GADD45B',
 'VWF',
 'LUM',
 'SERPINA1',
 'TACSTD2',
 'EPCAM',
 'SYNE4',
 'CTSS',
 'IGFBP4',
 'TNS2',
 'PKP3',
 'UQCRH',
 'XBP1',
 'BATF',
 'PAFAH1B3',
 'HEXA',
 'WFDC2',
 'TRIB3',
 'EFHD1',
 'PFN2',
 'STRAP',
 'MSR1',
 'VWA1',
 'DSTN',
 'LAPTM5',
 'KRT19',
 'SLC7A2',
 'COLEC12',
 'SERINC2',
 'XAF1',
 'IRX3',
 'PARD6B',
 'AHNAK',
 'TNFRSF18',
 'PARP9',
 'THY1',
 'CCDC80',
 'CITED4',
 'CORO1A',
 'NPC2']